In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from sklearn import model_selection, preprocessing

In [ ]:
from tensorflow.keras import activations, datasets, layers, losses, metrics, models, backend, regularizers
import tensorview as tv
import tensorflow as tf

In [ ]:
import lightgbm

In [ ]:
import folium

In [ ]:
import utm

## Load train and test data

In [ ]:
# train set
df_mess_train = pd.read_csv('mess_train_list.csv')

# test set
df_mess_test = pd.read_csv('mess_test_list.csv')

# position associated to train set
pos_train = pd.read_csv('pos_train_list.csv') 

In [ ]:
df_mess_train.head()

In [ ]:
df_mess_train['did'].unique().shape

In [ ]:
print(df_mess_train.shape)
df_mess_train.describe()

In [ ]:
pos_train.head()

In [ ]:
pos_train.describe()

## Prepare data

In [ ]:
# determine all Base stations that received at least 1 message
trainBs  = np.unique(df_mess_train['bsid'])
testBs   = np.unique(df_mess_test['bsid'])
listOfBs = np.union1d(trainBs, testBs) 
testOnlyBs = np.lib.arraysetops.setdiff1d(testBs, trainBs)

print(f"Number of stations: %d, test only %d" % (len(listOfBs), len(testOnlyBs)))

In [ ]:
df_mess_train['did'].unique().shape

# Affichage des données

## Passage en UTM

In [ ]:

def latlon_to_xy(lat, lon):
    """Conversion lat/lon en UTM"""
    x, y, utm_zone, utm_letter = utm.from_latlon(lat, lon)
    return x, y, utm_zone, utm_letter


def xy_to_latlon(x, y, utm_zone, utm_letter):
    """Conversion UTM en lat/lon"""
    lat, lon = utm.to_latlon(x, y, utm_zone, utm_letter)
    return lat, lon


pos_train[['x', 'y', 'utm_zone', 'utm_letter']] = pos_train.apply(lambda row: pd.Series(latlon_to_xy(row['lat'], row['lng'])),axis=1)
df_mess_train[['bs_x', 'bs_y', 'bs_utm_zone', 'bs_utm_letter']] = df_mess_train.apply(lambda row: pd.Series(latlon_to_xy(row['bs_lat'], row['bs_lng'])),axis=1)
df_mess_test[['bs_x', 'bs_y', 'bs_utm_zone', 'bs_utm_letter']] = df_mess_test.apply(lambda row: pd.Series(latlon_to_xy(row['bs_lat'], row['bs_lng'])),axis=1)

## Filtering on outliers


In [ ]:
print(f"Nombres de messages du jeu d'apprentissage: {len(df_mess_train.messid.unique())}");
print(f"Nombres de messages du jeu d'apprentissage sans les stations au dessus du 60eme parallèle : {len(df_mess_train[df_mess_train.bs_lat<60].messid.unique())}");

In [ ]:
print(f"Nombres de messages du jeu de: {len(df_mess_test.messid.unique())}");
print(f"Nombres de messages du jeu de test sans les stations au dessus du 60eme parallèle : {len(df_mess_test[df_mess_test.bs_lat<60].messid.unique())}");

### Analyse
On a 150 messages du jeu d'apprentissage et 95 du jeu de test qui refère uniquement les stations au dessus du 60eme parallèle.
Ces stations sont très éloignés des autres données et semblent donc être des données anomaliques.
En analysant les coordonnées des labels pour ces messages, ils s'avèrent que les devices sont tous situé dans la zone des autres stations.
Nous prenons donc le parti de modifier les coordonnées de ces stations au barycentre des différentes positions des devices qui ont communiqué uniquement avec ces stations.

In [ ]:
mask = ~df_mess_train[df_mess_train.bs_lat > 60]["messid"].isin(df_mess_train[df_mess_train.bs_lat < 60]["messid"])
messid_far = df_mess_train[df_mess_train.bs_lat > 60]["messid"][mask].unique()
df_contat = pd.concat([df_mess_train,pos_train], axis=1)
zone_19_stations = df_contat[df_contat["messid"].isin(messid_far)]
zone_19_coords = zone_19_stations.mean(axis=0)[["lat", "lng", "x","y","utm_zone"]]

mask_19 = df_mess_train["bs_utm_zone"]!=19
df_mess_train.bs_lat = df_mess_train.bs_lat.where(mask_19, other=zone_19_coords.lat)
df_mess_train.bs_lng = df_mess_train.bs_lng.where(mask_19, other=zone_19_coords.lng)
df_mess_train.bs_x = df_mess_train.bs_x.where(mask_19, other=zone_19_coords.x)
df_mess_train.bs_y = df_mess_train.bs_y.where(mask_19, other=zone_19_coords.y)
df_mess_train.bs_utm_zone = df_mess_train.bs_utm_zone.where(mask_19, other=zone_19_coords.utm_zone)
df_mess_train.bs_utm_letter = df_mess_train.bs_utm_letter.where(mask_19, other="S")


In [ ]:
mask_19 = df_mess_test["bs_utm_zone"]!=19
df_mess_test.bs_lat = df_mess_test.bs_lat.where(mask_19, other=zone_19_coords.lat)
df_mess_test.bs_lng = df_mess_test.bs_lng.where(mask_19, other=zone_19_coords.lng)
df_mess_test.bs_x = df_mess_test.bs_x.where(mask_19, other=zone_19_coords.x)
df_mess_test.bs_y = df_mess_test.bs_y.where(mask_19, other=zone_19_coords.y)
df_mess_test.bs_utm_zone = df_mess_test.bs_utm_zone.where(mask_19, other=zone_19_coords.utm_zone)
df_mess_test.bs_utm_letter = df_mess_test.bs_utm_letter.where(mask_19, other="S")


In [ ]:
df_mess_train

## Construction de la matrice des featues
Pour cette matrice, nous n'utiliserons qu'une partie des données et nous ferons une moyenne geometrique sur les coordonnées des devices

In [ ]:
def feat_mat_const(df, listOfBs, keepMax=5):
    """ Feature Matrix construction """
    
    aggCols = ['did', 'pivot_lat', 'pivot_lng']
    for i in range(keepMax):
        bsCols =['bs%d_deltalat' % i, 'bs%d_deltalng' % i, 'bs%d_rssi' % i] #, 'bs%d_nseq' % i 'bs%d_active' % i, 
        aggCols = aggCols + bsCols
        
    def aggregateBaseStations(groupBy):
        """ From a RSSI sorted DataFrameGroupBy
            create a dataframe with the 3 best BS 
        """
        
        did = groupBy['did'][0]
        bsSet = groupBy.iloc[:keepMax]
            
        # Barycentre par moyenne geometrique
        w_geom = np.exp(bsSet['rssi']) / np.sum(np.exp(bsSet['rssi']))
        
        x_geom = np.exp(np.sum(w_geom * np.log(bsSet['bs_x'])) / np.sum(w_geom))
        y_geom = np.exp(np.sum(w_geom * np.log(bsSet['bs_y'])) / np.sum(w_geom))
        
        # Barycentre par moyenne arithmétique
        w_arm = bsSet['rssi'] / np.sum(bsSet['rssi'])
        
        x_arm = np.average(bsSet['bs_x'], weights=w_arm)
        y_arm = np.average(bsSet['bs_y'], weights=w_arm)
        
        bss = []
        for i in range(keepMax):
            if len(bsSet) > i:
                b = bsSet.iloc[i]
                dx = b['bs_x'] - x_geom
                dy = b['bs_y'] - y_geom
                bss.append([dy, dx, b['rssi']])
            else:
                bss.append([0, 0, -1e3])
        return pd.DataFrame(np.concatenate([[did, y_geom, x_geom], np.array(bss).ravel()]).reshape(1, -1), 
                            columns=aggCols)
            
    
    # Keep at max keepMax base-stations per message
    df = df.groupby('messid'). \
        apply(lambda x: x.sort_values(['rssi'], ascending=False)). \
        reset_index(drop=True).groupby('messid').apply(aggregateBaseStations)
    
    return df

In [ ]:
def ground_truth_const(df_mess_train, pos_train):
    """ Ground truth construction """
    
    df = pd.concat([df_mess_train[['messid']], pos_train], axis=1)
    df_mean = df.groupby('messid').mean()

    return df_mean['y'], df_mean['x']

In [ ]:
df_feat = feat_mat_const(df_mess_train, listOfBs, 5)
print(df_feat.shape)
df_feat.head()

In [ ]:
ground_truth_y, ground_truth_x = ground_truth_const(df_mess_train, pos_train)
ground_truth_y.shape

In [ ]:
df_feat_red = df_feat.drop('did', axis=1)

## Evaluate helpers

In [ ]:
def vincenty_vec(vec_coord):
    """ Now using geodesic distance instead of Vincenty """
    vin_vec_dist = np.zeros(vec_coord.shape[0])
    if vec_coord.shape[1] != 4:
        print('ERROR: Bad number of columns (shall be = 4)')
    else:
        vin_vec_dist = [geodesic(v[0:2], v[2:]).meters for v in vec_coord]

    return vin_vec_dist

In [ ]:
def eval_geoloc(y_train_lat , y_train_lng, y_pred_lat, y_pred_lng):
    """ Evaluate distance error for each predicted point """
    vec_coord = np.array([y_train_lat , y_train_lng, y_pred_lat, y_pred_lng])
    err_vec = vincenty_vec(np.transpose(vec_coord))
    
    return err_vec

In [ ]:
def plotError(err_vec):
    """ Plot error cumulative distribution and the 80 quantile """
    
    err80 = np.percentile(err_vec, 80)
    
    print(f"error @ 80% = {err80:.1f} m")
    
    values, base = np.histogram(err_vec, bins=50000)
    cumulative = np.cumsum(values) 

    plt.figure()
    plt.plot(base[:-1]/1000, cumulative / np.float(np.sum(values))  * 100.0,
             label="Opt LLR", c='blue')

    # plot error @ 80%
    plt.axvline(x=err80/1000., ymin=0, ymax=100,
                linestyle='dashed', color='red')

    plt.xlabel('Distance Error (km)')
    plt.ylabel('Cum proba (%)')
    plt.axis([0, 30, 0, 100]) 

    plt.title('Error Cumulative Probability')
    plt.legend()

    plt.grid()

# Deep neural Network

In [ ]:
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(df_feat_red.values, 
                                                                np.c_[ground_truth_y.values, ground_truth_x.values], 
                                                                test_size=0.1)
# Normalize data to get proper network optimization
scaleX = preprocessing.StandardScaler()
scaleX.fit(xtrain)
xtrain = scaleX.transform(xtrain)
xtest = scaleX.transform(xtest)

scaleY = preprocessing.StandardScaler()
scaleY.fit(ytrain)
ytrain = scaleY.transform(ytrain)
# NO ytest = scaleY.transform(ytest)

In [ ]:
model1 = models.Sequential([
    layers.Dense(128, name='dense_1', activation=activations.relu, input_shape=[df_feat_red.shape[1]]),
    layers.Dropout(0.01),
    layers.Dense(32, name='dense_2', activation=activations.relu),
   #  layers.Dropout(0.01),
    layers.Dense(2, name='dense_3', activation=activations.linear),
])

model1.compile(optimizer='adam',
          loss=losses.MeanSquaredError())
    
model1.summary()

metricNames = ['Loss']

In [ ]:
nEpochs = 96
batchSize = 64

tvPlot = tv.train.PlotMetricsOnEpoch(metrics_name=metricNames,
                                      cell_size=(6,4), columns=2, iter_num=nEpochs, wait_num=1)

history1 = model1.fit(xtrain, ytrain,
            epochs=nEpochs, batch_size=batchSize, 
            validation_split=0.1, 
            verbose=0,
            callbacks=[tvPlot]);

In [ ]:
weights1 = model1.get_weights()
plt.hist(weights1[0].ravel(), bins=30);

In [ ]:
yEst = model1.predict(xtest)
yEst = scaleY.inverse_transform(yEst)

In [ ]:
metrics.mean_squared_error(ytest[:,0], yEst[:,0]).numpy(), \
metrics.mean_squared_error(ytest[:,1], yEst[:,1]).numpy()

In [ ]:
#dnnErr_vec = eval_geoloc(ytest[:,0], ytest[:,1], yEst[:,0].reshape(-1), yEst[:,1].reshape(-1))
#plotError(dnnErr_vec)

# The coordinates are in true X,Y for distance computation
delta_y = yEst[:,0] - ytest[:,0]
delta_x = yEst[:,1] - ytest[:,1]


dnnErr_vec = np.sqrt(delta_x**2 + delta_y**2)

plotError(dnnErr_vec)

## DNN with leave out device K fold validation

Utilisation de GroupKFold de Scikit Learn, génère K fold en évitant que les éléments d'un groupe se trouvent dans le train et la valid

In [ ]:
gkl1o = model_selection.GroupKFold(24) #model_selection.LeaveOneGroupOut()

In [ ]:
scaleX = preprocessing.StandardScaler()
scaleX.fit(df_feat_red)
df_feat_scaled = scaleX.transform(df_feat_red)

y = np.c_[ground_truth_y.values, ground_truth_x.values]
scaleY = preprocessing.StandardScaler()
scaleY.fit(y)
yscaled = scaleY.transform(y)

In [ ]:
i = 0
errors80Dnn = []
nEpochs = 96
for train, val in gkl1o.split(df_feat, groups=df_feat['did']):
    
    i+=1
    if i % 4 == 0: print('Split #', i)  
    
    xtrain = df_feat_scaled[train]
    xval = df_feat_scaled[val]
    ytrain = yscaled[train]
    yval = yscaled[val]
    
    model1.reset_states()
    model1.fit(xtrain, ytrain,
            epochs=nEpochs, batch_size=batchSize, 
            validation_data=(xval, yval),
            verbose=0)
    yEst = model1.predict(xval)
    yEst = scaleY.inverse_transform(yEst)
    
    delta_y = yEst[:,0] - ground_truth_y[val]
    delta_x = yEst[:,1] - ground_truth_x[val]
    
    dnnErr_vec = np.sqrt(delta_x**2 + delta_y**2)
    
    #dnnErr_vec = eval_geoloc(ground_truth_lat[val], ground_truth_lng[val], yEst[:,0], yEst[:,1])
    err80 = np.percentile(dnnErr_vec, 80)
    #print('Err @ 80%%, %.1fm' % (err80))
    errors80Dnn.append(err80) 

# LightGBM with Leave out device K fold

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 250,
    'learning_rate': 0.02,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

i = 0
didLgbm = []
errors80Lgbm = []
for train, val in gkl1o.split(df_feat, groups=df_feat['did']):
    i+=1
    if i % 4 == 0:
        print('Split #', i)
    
    xtrain = df_feat_red.values[train]
    xval = df_feat_red.values[val]
    
    lat_train = lightgbm.Dataset(xtrain, ground_truth_y[train])
    lat_valid = lightgbm.Dataset(xval, ground_truth_y[val])
    model_lat = lightgbm.train(params,
                           lat_train,
                           valid_sets=lat_valid,
                           num_boost_round=5000,
                           early_stopping_rounds=250,verbose_eval=False) 

    lng_train = lightgbm.Dataset(xtrain, ground_truth_x[train])
    lng_valid = lightgbm.Dataset(xval, ground_truth_x[val])
    model_lng = lightgbm.train(params,
                           lng_train,
                           valid_sets=lng_valid,
                           num_boost_round=5000,
                           early_stopping_rounds=250,verbose_eval=False) 

    # Evaluate
    lat_pred = model_lat.predict(xval)
    lng_pred = model_lng.predict(xval)
    
    delta_y = lat_pred - ground_truth_y[val]
    delta_x = lng_pred - ground_truth_x[val]
    
    lgbmErr_vec = np.sqrt(delta_x**2 + delta_y**2)
    #lgbmErr_vec = eval_geoloc(ground_truth_y[val], ground_truth_x[val], lat_pred, lng_pred)
    err80 = np.percentile(lgbmErr_vec, 80)
    errors80Lgbm.append(err80) 

In [ ]:
#lat_pred = model_lat.predict(xtest)
#lng_pred = model_lng.predict(xtest)
#
#lgbmErr_vec = eval_geoloc(ytest[:,0], ytest[:,1], lat_pred, lng_pred)
#plotError(lgbmErr_vec)

## Plot leave 1 device out performance

In [ ]:
errorMeanOverDevicesDnn = np.array(errors80Dnn).mean()
errorMeanOverDevicesLgbm = np.array(errors80Lgbm).mean()
print('Mean error over devices @ 80%%, DNN : %.1fm, LGBM : %.1fm' % \
          (errorMeanOverDevicesDnn, errorMeanOverDevicesLgbm))
fig, ax = plt.subplots(figsize=(15, 6))
#ax.hist(errors80Dnn,  bins=5, alpha=0.7, label='DNN')
ax.hist(errors80Lgbm, bins=10, alpha=0.7, label='LGBM')
ax.set_title('Histogram of error on the 1 device out')
ax.set_xlabel('Error@ 80%[m]')
#ax.axvline(x=errorMeanOverDevicesDnn,  color='blue', linestyle='--')
ax.axvline(x=errorMeanOverDevicesLgbm, color='orange', linestyle='--')
ax.legend()
ax.grid()

## Study outliers on the error 80

Device ID=476835 a des performances très mauvaises (erreur @ 80% > 100km).

Cette section détermine les problèmes des messages de ce device

In [ ]:
outlier1Did = 476835
outlier1Index = df_mess_train['did'] == outlier1Did
outlier1X = df_mess_train[outlier1Index]
outlier1X.head()

In [ ]:
outlier1MessId = df_mess_train[outlier1Index].reset_index()['messid'].unique()
outlier1MessId.shape

In [ ]:
df_feat_rank1OutlierIndex = df_feat['did']== outlier1Did
df_feat[df_feat_rank1OutlierIndex].head(5)

In [ ]:
outlier1y = y[df_feat_rank1OutlierIndex]

Utilisation du dernier estimateur LGBM

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)
error_lat = model_lat.predict(df_feat_red.loc[outlier1MessId]) - outlier1y[:,0]
axes[0].plot(error_lat)
axes[0].set_title('Rank 1 outlier latitude error')
axes[0].set_xlabel('message')
axes[0].set_ylabel('lat error [deg]')
axes[0].grid()
error_lng = model_lng.predict(df_feat_red.loc[outlier1MessId]) - outlier1y[:,1]
axes[1].plot(error_lng)
axes[1].set_title('Rank 1 outlier longitude error')
axes[1].set_xlabel('message')
axes[1].set_ylabel('lng error [deg]')
axes[1].grid()

In [ ]:
df_feat_red.loc[outlier1MessId][50:]

Les 17 derniers messages n'ont qu'une seule BS et le device s'éloigne de la BS.

Ils sont affichés en bleu sur la carte suivante, au sud de Denver. Les BS, de tous les messages de ce device, sont en rouge.

In [ ]:
m = folium.Map(
    location=[39.8, -105.0],
    zoom_start=6,
    tiles='Stamen Terrain'
)

for r in outlier1X.iterrows():
    row = r[1]
    folium.Circle(
        radius=max(5, 200+row['rssi']),
        location=[row['bs_lat'], row['bs_lng']],
        #popup='The Waterfront',
        color='crimson',
        fill=False,
    ).add_to(m)


for row in outlier1y[52:]:
    #row = r[1]
    lat, lon = xy_to_latlon(row[1], row[0], 13,'S')
    folium.Circle(
        radius=10,
        location=[lat, lon],
        #popup='The Waterfront',
        color='blue',
        fill=False,
    ).add_to(m)

m

## Construct test prediction

In [ ]:
df_mess_test.head()

In [ ]:
df_feat_test = feat_mat_const(df_mess_test, listOfBs)
df_feat.shape, df_feat_test.shape

In [ ]:
y_pred_test_lat, y_pred_test_lng, reg = regressor_and_predict(df_feat, ground_truth_lat, 
                                                    ground_truth_lng, df_feat_test, False)

In [ ]:
test_res = pd.DataFrame(np.array([y_pred_test_lat, y_pred_test_lng]).T, columns = ['lat', 'lng'])
test_res = pd.concat([df_mess_test['messid'], test_res], axis=1)

In [ ]:
test_res.head()

In [ ]:
test_res.to_csv('pred_pos_test_list.csv', index=False)